In [ ]:
import random

class EightQueens:
    def __init__(self, initial_state=None, mode='empty'):
        self.size = 8
        if initial_state:
            self.state = initial_state #[-1,-1,-1,-1]
        elif mode == 'random':
            self.state = random.sample(range(self.size), self.size)
        else:
            self.state = [-1] * self.size  # -1 means no queen placed
        self.variables = list(range(self.size))  # one variable per row
        self.domains = [list(range(self.size)) for _ in range(self.size)]  # columns for each queen

    def is_consistent(self, row, col):
        #Check if placing a queen at (row, col) does not conflict with previous queens.
        for r in range(row):
            c = self.state[r]
            if c == col or abs(row - r) == abs(col - c):
                return False
        return True

    def get_actions(self, row):
        #Return all valid columns for placing a queen in the given row.
        return [col for col in self.domains[row] if self.is_consistent(row, col)]

    def apply_action(self, row, col):
        #Place a queen at the specified position.
        self.state[row] = col

    def transition_model(self, row, col):
        #Return new state after placing a queen at (row, col).
        new_state = self.state[:]
        new_state[row] = col
        return new_state

    def heuristic(self, state = None):
        #Count the number of conflicting queen pairs.
        conflicts = 0
        if state == None:
            state = self.state
            
        for i in range(self.size):
            for j in range(i + 1, self.size):
                if state[i] == -1 or state[j] == -1:
                    continue
                if state[i] == state[j] or abs(i - j) == abs(state[i] - state[j]):
                    conflicts += 1
        return conflicts
    
    def cost(self, state):
        return sum(1 for col in state if col != -1)

    def print_board(self):
        #Display the board as ASCII.
        for r in range(self.size):
            row = ["Q" if self.state[r] == c else "." for c in range(self.size)]
            print(" ".join(row))
        print("Heuristic (conflicts):", self.heuristic(self.state))

In [14]:
def create_population(size=100):
    population = []
    for _ in range(size):
        state = EightQueens(mode='random')
        population.append(state)
        
    return population

def genetic_algorithm(population, mutation_probability:float = 0.5):
    new_population = []
    
    best_heuristic = float('inf')
    best_state = None
    
    for i in range(len(population)):
        parent1, parent2 = select_parents(population)
        
        merged = reproduce(parent1, parent2)
        
        if random.random() < mutation_probability:
            mutate(merged)
            
        new_population.append(merged)
        h = merged.heuristic()
        
        if h < best_heuristic:
            best_heuristic = h
            best_state = merged
    
    population = new_population        
    return best_state  

def reproduce(state1, state2):
    x = state1.state
    y = state2.state
    n = len(x)
    c = random.randint(1, n - 1)
    new_state = x[:c] + y[c:]
    return EightQueens(initial_state=new_state)


def mutate(state):
    row = random.randint(0, state.size - 1)
    new_col = random.randint(0, state.size - 1)
    state.state[row] = new_col
    
def select_parents(population):
    weights = [1 / (1 + individual.heuristic()) for individual in population]
    return random.choices(population, weights=weights, k=2)

In [ ]:
population = create_population(100)

print('Length of list: ')
print(len(population))


for i in range(1, 101):
    resulting_state = genetic_algorithm(population)

    print(f"{i}.")
    print('Length of list after Genetic Algorithm: ')
    print(len(population))

    resulting_state.print_board()
    print()